- **id** - id команды
- **api_id** - ключ на таблицу matches
- **long_name**	-    полное название команды
- **short_name** -	сокращённое название команды
 
 ### math

- **id** -	id матча
- **season**-	сезон
- **date** -	дата матча
- **home_team_api_id** -	api_id домашней команды, ключ на таблицу teams
- **away_team_api_id**	-api_id гостевой команды, ключ на таблицу teams
- **home_team_goals** -	количество голов домашней команды
- **away_team_goals** -	количество голов гостевой команды

Для **INNER JOIN** работает следующее правило: присоединяются только те строки таблиц, которые удовлетворяют условию соединения. Если в любой из соединяемых таблиц находятся такие строки, которые не удовлетворяют заявленному условию, — они отбрасываются.

Рассмотрим на примере нашего датасета. 

В таблице teams есть данные о 299 различных командах — можем проверить это с помощью запроса.

In [ ]:
"""
SELECT 
COUNT(DISTINCT api_id) /*оператор подсчёта строк; оператор исключения повторяющихся строк; столбец api_id*/
FROM sql.teams
"""
#Теперь добавим к teams таблицу с матчами.

"""
SELECT 
COUNT(DISTINCT t.id) /*оператор подсчёта строк; оператор исключения повторяющихся строк; столбец id*/
FROM 
sql.teams t /*таблица teams с алиасом t*/
JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id 
/*оператор соединения inner JOIN; таблица teams с алиасом t; условие: home_team_api_id 
таблицы m равен api_id таблицы t или away_team_api_id таблицы m равен api_id таблицы t*/
"""

И в таблице останется уже не 299 команд, а только 292.
Почему?

Дело в том, что таблица sql.matches по какой-то причине не содержит информацию о командах Lierse SK, KVC Westerlo, KAS Eupen, Club Brugge KV, KV Oostende, RSC Anderlecht и Hull City, зато они есть в таблице sql.teams. Возможно, эти команды не участвовали ни в одном матче или записи по этим матчам были удалены.

### LEFT OUTER JOIN И RIGHT OUTER JOIN
Также существуют схожие друг с другом типы соединения — LEFT JOIN и RIGHT JOIN (слово outer в операторе можно опустить).

Для LEFT JOIN работает следующее правило: из левой (относительно оператора) таблицы сохраняются все строки, а из правой добавляются только те, которые соответствуют условию соединения. Если в правой таблице не находится соответствия, то значения строк второй таблицы будут иметь значение NULL.

LEFT JOIN может быть полезен, когда соответствующих записей во второй таблице может не быть, но важно сохранить записи из первой таблицы.

?
Почему соответствий может не быть?

Причины могут быть разные, назовём две основные:

1
Различная бизнес-логика таблиц.

Пример: в интернет-магазине почти всегда можно оформить заказ с промокодом. Вполне вероятно, что информация о промокодах хранится в отдельной таблице, но при этом не для каждого заказа будет существовать промокод, поскольку покупку можно оформить и без него.

2
Разное время обновления таблиц.

Пример: клиент зарегистрировался на сайте интернет-магазина и оформил заказ. Данные о регистрации клиентов могут обновляться в базе данных раз в неделю, а о заказах — раз в сутки.

→ Поставим следующую задачу: вывести полные названия команд, данных по которым нет в таблице matches.

Для начала посмотрим на результат запроса после соединения.

In [ ]:
#Обратите внимание! Если мы добавим какой-либо фильтр по значению для таблицы matches, то LEFT JOIN превратится в INNER JOIN, поскольку для второй таблицы станет необходимым присутствие значения в строке.
"""
SELECT
    t.long_name 
FROM sql.teams t
LEFT JOIN sql.matches m ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id
WHERE
    m.season = '2008/2009'
AND t.long_name = 'KAS Eupen'
"""

In [ ]:
#/*Используя LEFT JOIN, выведите список уникальных названий команд, 
#содержащихся в таблице matches. Отсортируйте список в алфавитном порядке..
"""
SELECT
    distinct t.long_name
FROM sql.matches m /*таблица teams с алиасом t*/
LEFT JOIN  sql.teams t ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id 
order by t.long_n
"""

In [ ]:
#/*Используя LEFT JOIN, напишите запрос, который выведет полное название команды (long_name), 
#количество матчей, в которых участвовала команда, — домашних и гостевых (matches_cnt). 
#Отсортируйте по количеству матчей в порядке возрастания, затем — по названию команды в алфавитном порядке.
#*/
"""
SELECT
    long_name,
    count(m.id) matches_cnt
FROM  sql.teams t
LEFT JOIN sql.matches m  ON t.api_id = m.home_team_api_id OR t.api_id = m.away_team_api_id 
group by t.id
order by count(t.long_name), t.long_name
"""

In [ ]:
#В начале модуля вы использовали этот оператор, записывая таблицы через запятую.
"""
SELECT *
FROM
    sql.teams,
    sql.matches
"""
#Этот же запрос можно записать с использованием CROSS JOIN.
"""
SELECT *
FROM
    sql.teams
    CROSS JOIN sql.matches
"""
#Обратите внимание! Условие для CROSS JOIN, в отличие от других операторов, не требуется.

#Также этот запрос можно записать с помощью INNER JOIN с условием on true — результат будет тот же.


"""
SELECT *
FROM
    sql.teams
    JOIN sql.matches ON TRUE
"""
#CROSS JOIN может быть полезен, когда необходимо создать таблицу фактов.

#Например, с помощью такого запроса мы можем получить все возможные комбинации полных названий команд в матчах.

"""
SELECT
    DISTINCT
    t1.long_name home_team, 
    t2.long_name away_team
FROM
    sql.teams t1
    CROSS JOIN sql.teams t2
"""

NATURAL JOIN

Ключевое слово natural в начале оператора JOIN позволяет не указывать условие соединения таблиц — для соединения будут использованы столбцы с одинаковым названием из этих таблиц.

NATURAL JOIN можно использовать с любыми видами соединений, которые требуют условия соединения:

- NATURAL INNER JOIN (возможна запись NATURAL JOIN);
- NATURAL LEFT JOIN;
- NATURAL RIGHT JOIN;
- NATURAL FULL OUTER JOIN.

При использовании NATURAL JOIN прежде всего стоит обратить внимание на ключи таблиц. Для наших таблиц teams и matches этот вид соединения не подойдёт, так как общим для обеих таблиц является столбец id, но таблицы соединяются по другим столбцам.

Когда у таблиц есть несколько столбцов с одинаковыми именами, при NATURAL JOIN условие соединения будет применено на все столбцы с одинаковыми именами.

То есть для таблиц table1 и table2

In [ ]:
"""
SELECT 
…
FROM 
         table1 
NATURAL JOIN table2
"""
#будет равнозначен запросу
"""
SELECT
…
FROM 
         table1 t1
INNER JOIN table2 t2 ON t1.id = t2.id AND t1.name = t2.name
"""

In [ ]:
#/*Напишите запрос, который выведет полное название команды (long_name) 
#и общее количество матчей (matches_cnt), сыгранных командой Inter в домашних матчах.
"""
SELECT
    long_name,
    count(m.id) matches_cnt
FROM  sql.teams t
LEFT JOIN sql.matches m  ON t.api_id = m.home_team_api_id 
where long_name = 'Inter'
group by t.id
order by count(t.long_name), t.long_name
"""

In [ ]:
#/*Напишите запрос, который выведет топ-10 команд (long_name) по суммарному количеству забитых голов в гостевых матчах. 
#Во втором столбце запроса выведите суммарное количество голов в гостевых матчах (total_goals).
#*/
"""
SELECT
    long_name,
    sum(a.away_team_goals) as total_goals
FROM  sql.teams t
    JOIN sql.matches a  ON t.api_id = a.away_team_api_id 
group by t.id
order by sum(a.away_team_goals) desc
limit 10
"""

In [ ]:
#/*Выведите количество матчей между командами Real Madrid CF и FC Barcelona. 
#В поле ниже введите запрос, с помощью которого вы решили задание.
#*/
"""
SELECT
    count(m.id)
FROM  sql.matches m 
    left JOIN sql.teams a  ON a.api_id = m.away_team_api_id
    left JOIN sql.teams h  ON h.api_id = m.home_team_api_id
where  (h.long_name = 'Real Madrid CF' or h.long_name = 'FC Barcelona') and (a.long_name = 'Real Madrid CF' or a.long_name = 'FC Barcelona')

"""


In [ ]:
#/*Напишите запрос, который выведет название команды (long_name), 
#сезон (season) и суммарное количество забитых голов в домашних матчах (total_goals). 
#Оставьте только те строки, в которых суммарное количество голов менее десяти. 
#Отсортируйте запрос по названию команды, а затем — по сезону.
#*/
"""
SELECT
    t.long_name,
    season,
    sum(a.home_team_goals) as total_goals
FROM  sql.matches a
    left JOIN sql.teams t  ON t.api_id = a.home_team_api_id 
group by t.api_id,t.long_name,a.season
having sum(a.home_team_goals) < 10
order by t.long_name, season
"""
